In [1]:
# para rodar no colab
#!pip uninstall networkx -y
#!pip install networkx==2.3
import networkx
networkx.__version__

#Adicionando biblioteca "nx@networkx" para gerenciamento e produção de graficos 
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.cm as cm
import math as mt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import random
import warnings
import networkx.algorithms.community as nx_comm
import os as os

from numpy  import *
from scipy.linalg import expm
from scipy.stats import moment
from scipy.stats import entropy
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from networkx.algorithms import community
from networkx.algorithms.community import LFR_benchmark_graph
from sklearn.metrics import normalized_mutual_info_score
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import label_propagation_communities
#from community import community_louvain
#from google.colab import drive

warnings.filterwarnings('ignore')
np.random.seed(50)
#drive.mount('/content/drive/')
#definindo dimesões padrão para plot
plt.rcParams['figure.figsize'] = (11,7)

def plot_grafo(grafo):
    plots = nx.spring_layout(grafo)
    nx.draw(grafo, with_labels = True, node_size=1, font_size=10, pos = plots)
    plt.show(True)

# **Questão 1**

***Bonus***

Implement the model of complex networks with assortativity by
Xulvi-Brunet-Sokolov and Verify how the assortativity influences the network
resilience against failures and attacks. That is, construct the plots of the
size of the largest component in terms of the fraction of removed nodes
for assortativity values: -0.2, 0 and 0.2.

# **Questão 2**

# **Questão 3**

Construct the plot of the fraction of removed nodes (in the SIR dynamics)
versus a centrality measure (degree, betweenness centrality, closeness
centrality, eigenvector centrality, accessibility, k-core and
communicability centrality). Verify which of these measures most
influence the fraction of removed nodes.

**a)** Hamsterster friendships

**b)** C. elegans neural network

**c)** US airport network

# **Bonus**

***Verify how the assortativity influences the epidemic
spreading in complex networks for the SIR model.***